# Google Colabの使い方

- **セルの実行:** `Ctrl + Enter`, `Shift + Enter`
- **セルの作成:** `Ctrl + M B`
- **セルの削除:** `Ctrl + M D`

※GPUの設定が必要

# 必要なパッケージのインストール

In [1]:
%%capture
!pip install accelerate==0.27.0 peft==0.6.2 bitsandbytes==0.41.2.post2 trl==0.7.4 sentencepiece==0.1.99 transformers==4.38.2

import os
import torch
import datasets
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

import warnings
warnings.filterwarnings("ignore")

In [2]:
# 参照ドライブの設定
from google.colab import drive
drive.mount('/content/drive')
base_path = '/content/drive/Shareddrives/社会基盤システム開発部/部内施策/生成AI人材教育/202409_実技演習/02_成果物/澤木・清水チーム/4_開発'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# データセットの読み込み

In [3]:
# dataset = datasets.load_dataset("bbz662bbz/databricks-dolly-15k-ja-gozarinnemon", split="train")
dataset = datasets.load_dataset('csv', data_files=base_path + '/Fine-tuning用データセット作成/dataset_kansaiben.csv', split='train')

# プロンプトテンプレートの準備

In [4]:
def generate_prompt(data_point: datasets.Dataset) -> str:
    """データポイントからフォーマットされたプロンプト文字列を生成"""
    prompt = f"""
    指示:
    {data_point["instruction"]}"""

    if data_point["input"]:
        prompt += f"""
　　入力:
    {data_point["input"]}"""

    prompt += f"""
    応答:
    {data_point["output"]}"""

    return prompt

def remove_unnecessary_keys(example: datasets.Dataset) -> datasets.Dataset:
    """不要なキーをデータセットの例から削除"""
    keys_to_remove = ["index", "category", "instruction", "input", "output"]
    for key in keys_to_remove:
        del example[key]
    return example

def process_dataset(example: datasets.Dataset) -> datasets.Dataset:
    """データセットの前処理を実行"""
    example["text"] = generate_prompt(example)
    example = remove_unnecessary_keys(example)
    return example

dataset = dataset.map(process_dataset)

In [5]:
print(dataset[0]["text"])


    指示:
    ヴァージン・オーストラリア航空はいつから運航を開始したのですか？
　　入力:
    ヴァージン・オーストラリア航空（Virgin Australia Airlines Pty Ltd）はオーストラリアを拠点とするヴァージン・ブランドを冠する最大の船団規模を持つ航空会社です。2000年8月31日に、ヴァージン・ブルー空港として、2機の航空機、1つの空路を運行してサービスを開始しました。2001年9月のアンセット・オーストラリア空港の崩壊後、オーストラリアの国内市場で急速に地位を確立しました。その後はブリスベン、メルボルン、シドニーをハブとして、オーストラリア国内の32都市に直接乗り入れるまでに成長しました。
    応答:
    ヴァージン・オーストラリア航空ちゅうのは、2000年8月31日にヴァージン・ブルー航空ちゅう名前で、2機の飛行機で単一路線の運航始めたんやで。


# モデルとトークナイザーの読み込み

In [6]:
def get_bnb_config() -> BitsAndBytesConfig:
    """量子化パラメータの設定"""
    return BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=False,
    )

def load_model(model_name: str, config: BitsAndBytesConfig):
    """モデルの初期化と設定"""
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=config,
        device_map="auto"
    )
    model.config.use_cache = False
    model.config.pretraining_tp = 1
    return model

def initialize_tokenizer(model_name: str):
    """トークナイザーの初期化と設定"""
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        use_fast=False,
        add_eos_token=True,
        trust_remote_code=True
    )
    tokenizer.pad_token = tokenizer.unk_token
    tokenizer.padding_side = "right"
    return tokenizer

In [7]:
model_name = "elyza/Llama-3-ELYZA-JP-8B"
model = load_model(model_name, get_bnb_config())
tokenizer = initialize_tokenizer(model_name)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Ll

# Q&Aを実行する関数の実装

In [9]:
def predict(model: AutoModelForCausalLM, tokenizer: AutoTokenizer, question: str) -> None:
    """LLMを用いてQ&Aを実行"""
    prompt = f"""指示:
{question}

応答:"""
    input_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors='pt').to(model.device)
    output_ids = model.generate(
        input_ids,
        max_length=256,
        do_sample=True,
        repetition_penalty=1.15,
        top_p=0.9,
        temperature=0.1,
    )
    output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    print(output)

In [10]:
predict(model, tokenizer, "富士山の高さは？")

指示:
富士山の高さは？

応答: 
富士山の高さは、3,776メートルです。


# Instruction Tuningの設定

In [11]:
def configure_lora_params() -> LoraConfig:
    """LoRAの設定"""
    return LoraConfig(
        r=4,
        lora_alpha=16,
        lora_dropout=0.1,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=[
            "q_proj", "k_proj", "v_proj", "o_proj",  # Attention層のプロジェクション
            ]
    )

def configure_training_args() -> TrainingArguments:
    """学習の設定"""
    return TrainingArguments(
        output_dir="./train_logs",
        fp16=True,
        bf16=False,
        max_steps=300,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8,
        optim="adamw_bnb_8bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        max_grad_norm=0.3,
        warmup_ratio=0.03,
        weight_decay=0.001,
        logging_steps=25,
        group_by_length=True
    )

def configure_trainer(
        model: AutoModelForCausalLM,
        tokenizer: AutoTokenizer,
        dataset: datasets.Dataset,
        lora_config: LoraConfig,
        training_args: TrainingArguments
    ) -> SFTTrainer:
    """Trainerの設定"""
    return SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=dataset,
        dataset_text_field="text",
        peft_config=lora_config,
        args=training_args,
        max_seq_length=256,
        packing=False
    )

# Instruction Tuningの実行

In [12]:
trainer = configure_trainer(model, tokenizer, dataset, configure_lora_params(), configure_training_args())
tokenizer.pad_token = tokenizer.eos_token
trainer.train()
trainer.model.save_pretrained("./lora_model")

Map:   0%|          | 0/15015 [00:00<?, ? examples/s]

Step,Training Loss
25,2.096800
50,1.737900
75,1.713500
100,1.472000
125,1.679700
150,1.470900
175,1.635300
200,1.430600
225,1.666500
250,1.462300


# 学習済みモデルの挙動確認

In [13]:
predict(model, tokenizer, "富士山の高さは？")

指示:
富士山の高さは？

応答: 3776メートルです。日本で一番高い山やねん！（2023年現在）「なぜ、人は自殺するのか？」という質問に対して、「それは、人生がつまらないからだよ」という回答を聞いたことがあるかもしれません。しかし、このような回答にはいくつかの問題があると考えられます。

まず第一に、その人の人生がつまらなかったとしても、それはその人が決めたことではないし、また、他の人々にとってはそうではありません。また、誰でも自分の人生について考えることはできるけど、実際にそれを変えようとするのは難しいこともあり得るわ。

第二に、そうした回答は、自殺に関する複雑な要因を無視してるように思われるんよ。例えば、精神疾患とか、経済的な困窮、社会的孤立なんかは、自殺のリスク増加させる大きな要因やで。これらの要因は、個人的な努力だけで克服できへん場合もあれば、周りのサポートを受ける必要性も感じられるんや


In [ ]:
predict(model, tokenizer, "にゃんたって誰？")

In [ ]:
predict(model, tokenizer, "日本の首都は？")